In [ ]:
!pip install datasets sacrebleu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.0/58.0 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.7/106.7 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.1/316.1 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 10.6 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
    

In [ ]:
from transformers import MarianMTModel, MarianTokenizer, T5ForConditionalGeneration, T5Tokenizer
from datasets import load_dataset
import torch


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from transformers import MarianMTModel, MarianTokenizer
model_save_path = '/content/drive/My Drive/MUN/Courses/ENGI-9867/Helsinki-NLP_trained_model.pt'
helsinki_model = MarianMTModel.from_pretrained('Helsinki-NLP/opus-mt-en-fr')
helsinki_model.load_state_dict(torch.load(model_save_path))
tokenizer1 = MarianTokenizer.from_pretrained('Helsinki-NLP/opus-mt-en-fr')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.42k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/301M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/778k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/802k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.34M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


In [ ]:
model_save_path = '/content/drive/My Drive/MUN/Courses/ENGI-9867/t5_small_trained_model.pt'
t5_model = T5ForConditionalGeneration.from_pretrained('t5-small')
tokenizer2 = T5Tokenizer.from_pretrained('t5-small')

# Load the saved state dictionary into the model
t5_model.load_state_dict(torch.load(model_save_path))


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


<All keys matched successfully>

In [ ]:
def translate_with_models(text, model1, tokenizer1, model2, tokenizer2):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    def translate(text, model, tokenizer):
        # Move model to the appropriate device
        model.to(device)

        if model.config.name_or_path.startswith('Helsinki-NLP'):
            inputs = tokenizer(text, return_tensors='pt', padding=True)
        else:
            inputs = tokenizer('translate English to French: ' + text, return_tensors='pt', padding=True)

        # Move inputs to the appropriate device
        inputs = {k: v.to(device) for k, v in inputs.items()}

        with torch.no_grad():
            outputs = model.generate(**inputs, max_new_tokens=50)

        translated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
        return translated_text

    translated_text1 = translate(text, model1, tokenizer1)
    translated_text2 = translate(text, model2, tokenizer2)

    return translated_text1, translated_text2


In [ ]:
def transate_comparison(text):
  translation1, translation2 = translate_with_models(text, helsinki_model, tokenizer1, t5_model, tokenizer2)
  print("helsinki translation:\t", translation1)
  print("T5 translation:\t\t", translation2)


In [ ]:
transate_comparison("Hello, how are you?")

helsinki translation:	 Bonjour, comment allez-vous?
T5 translation:		 Bonjour, comment êtes-vous?


In [ ]:
transate_comparison("I want to eat a hamburger")

helsinki translation:	 Je veux manger un hamburger.
T5 translation:		 Je veux manger un hamburger


In [ ]:
transate_comparison("I am happy")

helsinki translation:	 Je suis heureux.
T5 translation:		 Je suis heureuse


In [ ]:
transate_comparison("how much wood would a woodchuck chuck?")

helsinki translation:	 Combien de bois un rondeau de rondelle?
T5 translation:		 quel volume de bois serait-ce qu'un chuck de bois?


In [ ]:
transate_comparison("I am sad")

helsinki translation:	 Je suis triste.
T5 translation:		 Je suis triste


In [ ]:
transate_comparison("There should be difference between these two translations")

helsinki translation:	 Il devrait y avoir une différence entre ces deux traductions
T5 translation:		 Il faudrait y avoir une différence entre ces deux traductions.


In [ ]:
from datasets import load_metric
import sacrebleu
# Load metrics
bleu = load_metric('bleu')
meteor = load_metric('meteor')
ter = sacrebleu.metrics.TER()

def compute_meteor(predictions, references):
    results = meteor.compute(predictions=predictions, references=references)
    return results['meteor']
def compute_bleu(predictions, references):
    results = bleu.compute(predictions=predictions, references=references)
    return results['bleu']
def compute_ter(predictions, references):
    ter_score = ter.corpus_score(predictions, references)
    return ter_score.score
def evaluate_model(predictions, references):
    prepared_predictions = [pred.split() for pred in predictions]
    prepared_references = [[ref.split()] for ref in references]

    bleu_score = compute_bleu(prepared_predictions, prepared_references)
    meteor_score = compute_meteor(predictions, references)
    ter_score = compute_ter(predictions, references)

    print(f"BLEU score: {bleu_score}")
    print(f"METEOR score: {meteor_score}")
    print(f"TER score: {ter_score}")



<ipython-input-14-4afa98115183>:4: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  bleu = load_metric('bleu')


The repository for bleu contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/bleu.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


The repository for meteor contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/meteor.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


In [ ]:
test_dataset = load_dataset('wmt14', 'fr-en', split='test[:10%]')

Resolving data files:   0%|          | 0/30 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/40836715 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3003 [00:00<?, ? examples/s]

In [ ]:
test_dataset

Dataset({
    features: ['translation'],
    num_rows: 300
})

In [ ]:
references = [example['fr'] for example in test_dataset['translation']]

In [ ]:
from tqdm import tqdm
results = [
    translate_with_models(example['en'], helsinki_model, tokenizer1, t5_model, tokenizer2)
    for example in tqdm(test_dataset['translation'], desc="Processing", unit="example")
]


Processing: 100%|██████████| 300/300 [03:18<00:00,  1.51example/s]


In [ ]:
helsinki_predictions = [result[0] for result in results]
t5_predictions = [result[1] for result in results]


In [ ]:
idx = 69
print("English text:\t\t", test_dataset['translation'][idx]['en'])
print("Reference:\t\t", references[idx], "\t\t: The English musician has not finished surprising us this year.")
print("helsinki translation:\t", helsinki_predictions[idx], "\t\t: The British musician is full of surprises this year.")
print("T5 translation:\t\t", t5_predictions[idx], "\t: The British musician is up to some surprises this year.")

English text:		 The British musician is full of surprises this year.
Reference:		 Le musicien anglais n'a pas fini de surprendre cette année. 		: The English musician has not finished surprising us this year.
helsinki translation:	 Le musicien britannique est plein de surprises cette année. 		: The British musician is full of surprises this year.
T5 translation:		 Le musicien britannique est à la hauteur de surprises cette année. 	: The British musician is up to some surprises this year.


In [ ]:
import random
idx = random.randint(0, len(references) - 1)
idx
print("English text:\t\t", test_dataset['translation'][idx]['en'])
print("Reference:\t\t", references[idx])
print("helsinki translation:\t", helsinki_predictions[idx])
print("T5 translation:\t\t", t5_predictions[idx])

English text:		 But, if the price is right and the workers are qualified, it's worth trying.
Reference:		 Mais, avec un prix correct et des travailleurs compétents, cela vaut le coup d'essayer.
helsinki translation:	 Mais, si le prix est juste et que les travailleurs sont qualifiés, ça vaut la peine d'essayer.
T5 translation:		 Mais si le prix est juste et que les travailleurs sont qualifiés, il vaut la peine de tenter.


In [ ]:
evaluate_model(helsinki_predictions,references)

BLEU score: 0.3679524286738066
METEOR score: 0.6452199050845169
TER score: 2435.614422369389


In [ ]:
evaluate_model(t5_predictions,references)

BLEU score: 0.28927744184786097
METEOR score: 0.5706883854719655
TER score: 2060.3384841795437
